In [2]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from tqdm import tqdm
import time, re

llm = ChatOllama(
    model="llama3.1:latest",
    temperature=0.0,
    base_url="http://localhost:11434",
    api_key="ollama"
)

In [3]:
llm.invoke(
    [
        HumanMessage(
            content="What is the capital of France?"
        )
    ]
).content  

'The capital of France is Paris.'

## Add addition tool

In [13]:
from langchain_core.tools import tool
import re

@tool
def sum_numbers_from_text(inputs: str) -> float:
    """
    Adds a list of numbers provided in the input string.
    
    Args:
        text: A string containing numbers that should be extracted and summed.
        
    Returns:
        The sum of all numbers found in the input.
    """
    # Use regular expressions to extract all numbers from the input
    numbers = [int(num) for num in re.findall(r'\d+', inputs)]
    result = sum(numbers)
    return result

In [14]:
sum_numbers_from_text.invoke("Add the numbers 10, 20, 30, and 40 and twenty together.")

100

In [19]:
from langgraph.prebuilt import create_react_agent

agent_exec = create_react_agent(model=llm, tools=[sum_numbers_from_text])
msgs = agent_exec.invoke({"messages": [("human", "Add the numbers -10, -20, -30")]})
msgs

# Correctly iterate through the message objects in the 'messages' list
for msg in msgs['messages']:
    print(msg.content) 

Add the numbers -10, -20, -30

60
The sum of the numbers -10, -20, and -30 is 60.


## subtraction tool

In [20]:
@tool
def subtract_numbers(inputs: str) -> dict:
    """
    Extracts numbers from a string, negates the first number, and successively subtracts 
    the remaining numbers in the list.

    This function is designed to handle input in string format, where numbers are separated 
    by spaces, commas, or other delimiters. It parses the string, extracts valid numeric values, 
    and performs a step-by-step subtraction operation starting with the first number negated.

    Parameters:
    - inputs (str): 
      A string containing numbers to subtract. The string may include spaces, commas, or 
      other delimiters between the numbers.

    Returns:
    - dict: 
      A dictionary containing the key "result" with the calculated difference as its value. 
      If no valid numbers are found in the input string, the result defaults to 0.

    Example Input:
    "100, 20, 10"

    Example Output:
    {"result": -130}

    Notes:
    - Non-numeric characters in the input are ignored.
    - If the input string contains only one valid number, the result will be that number negated.
    - Handles a variety of delimiters (e.g., spaces, commas) but does not validate input formats 
      beyond extracting numeric values.
    """
    # Extract numbers from the string
    numbers = [int(num) for num in inputs.replace(",", "").split() if num.isdigit()]

    # If no numbers are found, return 0
    if not numbers:
        return {"result": 0}

    # Start with the first number negated
    result = -1 * numbers[0]

    # Subtract all subsequent numbers
    for num in numbers[1:]:
        result -= num

    return {"result": result}

In [23]:
print("Args: \n", subtract_numbers.args) 

Args: 
 {'inputs': {'title': 'Inputs', 'type': 'string'}}


In [25]:
subtract_numbers.invoke("Subtract the numbers 100, 20, and 10 from each other.")

{'result': -130}

## Add multiplication and division tools

In [29]:
# Multiplication Tool
@tool
def multiply_numbers(inputs: str) -> dict:
    """
    Extracts numbers from a string and calculates their product.

    Parameters:
    - inputs (str): A string containing numbers separated by spaces, commas, or other delimiters.

    Returns:
    - dict: A dictionary with the key "result" containing the product of the numbers.

    Example Input:
    "2, 3, 4"

    Example Output:
    {"result": 24}

    Notes:
    - If no numbers are found, the result defaults to 1 (neutral element for multiplication).
    """
    # Extract numbers from the string
    numbers = [int(num) for num in inputs.replace(",", "").split() if num.isdigit()]
    print(numbers)

    # If no numbers are found, return 1
    if not numbers:
        return {"result": 1}

    # Calculate the product of the numbers
    result = 1
    for num in numbers:
        result *= num
        print(num)

    return {"result": result}


multiply_numbers.invoke("Multiply the numbers 2, 3, and 4 together.")

[2, 3, 4]
2
3
4


{'result': 24}

In [32]:
# Division Tool
@tool
def divide_numbers(inputs: str) -> dict:
    """
    Extracts numbers from a string and calculates the result of dividing the first number 
    by the subsequent numbers in sequence.

    Parameters:
    - inputs (str): A string containing numbers separated by spaces, commas, or other delimiters.

    Returns:
    - dict: A dictionary with the key "result" containing the quotient.

    Example Input:
    "100, 5, 2"

    Example Output:
    {"result": 10.0}

    Notes:
    - If no numbers are found, the result defaults to 0.
    - Division by zero will raise an error.
    """
    # Extract numbers from the string
    numbers = [int(num) for num in inputs.replace(",", "").split() if num.isdigit()]


    # If no numbers are found, return 0
    if not numbers:
        return {"result": 0}

    # Calculate the result of dividing the first number by subsequent numbers
    result = numbers[0]
    for num in numbers[1:]:
        result /= num

    return {"result": result}

divide_numbers.invoke("Divide the numbers 100, 5, and 2 and zero together.")

{'result': 10.0}

## Build a simple agent that can use all the tools

In [34]:
tools = [sum_numbers_from_text,subtract_numbers, multiply_numbers, divide_numbers]
tools

[StructuredTool(name='sum_numbers_from_text', description='Adds a list of numbers provided in the input string.\n\nArgs:\n    text: A string containing numbers that should be extracted and summed.\n\nReturns:\n    The sum of all numbers found in the input.', args_schema=<class 'langchain_core.utils.pydantic.sum_numbers_from_text'>, func=<function sum_numbers_from_text at 0x746bdfab51c0>),
 StructuredTool(name='subtract_numbers', description='Extracts numbers from a string, negates the first number, and successively subtracts \nthe remaining numbers in the list.\n\nThis function is designed to handle input in string format, where numbers are separated \nby spaces, commas, or other delimiters. It parses the string, extracts valid numeric values, \nand performs a step-by-step subtraction operation starting with the first number negated.\n\nParameters:\n- inputs (str): \n  A string containing numbers to subtract. The string may include spaces, commas, or \n  other delimiters between the nu

In [35]:
from langgraph.prebuilt import create_react_agent

# Create the agent with all tools
math_agent = create_react_agent(
    model=llm,
    tools=tools,
    # Optional: Add a system message to guide the agent's behavior
    prompt="You are a helpful mathematical assistant that can perform various operations. Use the tools precisely and explain your reasoning clearly."
)

In [49]:
response = math_agent.invoke({
    "messages": [("human", "What is 28 divided by 4 minus 20?")]
})

# Get the final answer
final_answer = response["messages"][-1].content
print(final_answer)
print_agent_messages(response)

To find the result of the expression "28 divided by 4 minus 20", we first perform the division operation. 28 divided by 4 is equal to 7.

Then, we subtract 20 from the result. So, 7 minus 20 equals -13.

Therefore, the final answer is -13.

Message 1: HumanMessage
Content: What is 28 divided by 4 minus 20?
Tool Name: None

Message 2: AIMessage
Tool Calls: 1
  Tool Call 1:
    Name: subtract_numbers
    Args: {'inputs': '28, -4, 20'}
    ID: 9a7b18c6-7266-4c0e-9394-f7f31e75a74e
Tool Name: None
Model: llama3.1:latest
Duration: 543.53ms

Message 3: ToolMessage
Content: {"result": -48}
Tool Name: subtract_numbers
Tool Call ID: 9a7b18c6-7266-4c0e-9394-f7f31e75a74e
Status: success

Message 4: AIMessage
Content: To find the result of the expression "28 divided by 4 minus 20", we first perform the division operation. 28 divided by 4 is equal to 7.

Then, we subtract 20 from the result. So, 7 minus 20 equals -13.

Therefore, the final answer is -13.
Tool Name: None
Model: llama3.1:latest
Durati

## Add pretty prints for all messages and tool calls

In [44]:
def print_agent_messages(response):
    """
    Print all messages from an agent response with proper formatting.
    Handles HumanMessage, AIMessage, and ToolMessage types.
    """
    messages = response.get('messages', [])
    
    for i, msg in enumerate(messages, 1):
        print(f"\n{'='*50}")
        print(f"Message {i}: {type(msg).__name__}")
        print('='*50)
        
        # Handle different message types
        if hasattr(msg, 'content') and msg.content:
            print(f"Content: {msg.content}")
        
        # For AIMessage, check for tool calls
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            print(f"Tool Calls: {len(msg.tool_calls)}")
            for j, tool_call in enumerate(msg.tool_calls, 1):
                print(f"  Tool Call {j}:")
                print(f"    Name: {tool_call.get('name', 'N/A')}")
                print(f"    Args: {tool_call.get('args', 'N/A')}")
                print(f"    ID: {tool_call.get('id', 'N/A')}")
        
        # For ToolMessage, show tool-specific info
        if hasattr(msg, 'name'):
            print(f"Tool Name: {msg.name}")
        if hasattr(msg, 'tool_call_id'):
            print(f"Tool Call ID: {msg.tool_call_id}")
        if hasattr(msg, 'status'):
            print(f"Status: {msg.status}")
        
        # Show additional metadata if available
        if hasattr(msg, 'response_metadata') and msg.response_metadata:
            metadata = msg.response_metadata
            if 'model' in metadata:
                print(f"Model: {metadata['model']}")
            if 'total_duration' in metadata:
                print(f"Duration: {metadata['total_duration'] / 1000000:.2f}ms")

# Test the function with the previous response



Message 1: HumanMessage
Content: What is 25 divided by 4 plus 20?
Tool Name: None

Message 2: AIMessage
Tool Calls: 2
  Tool Call 1:
    Name: divide_numbers
    Args: {'inputs': '25, 4'}
    ID: 32c671a2-f896-43cd-acf7-4bc4d8453c07
  Tool Call 2:
    Name: add_numbers
    Args: {'inputs': 'result of previous operation, 20'}
    ID: a30f8b02-e021-4c2d-898d-2f5624643c4e
Tool Name: None
Model: llama3.1:latest
Duration: 1381.07ms

Message 3: ToolMessage
Content: {"result": 6.25}
Tool Name: divide_numbers
Tool Call ID: 32c671a2-f896-43cd-acf7-4bc4d8453c07
Status: success

Message 4: ToolMessage
Content: Error: add_numbers is not a valid tool, try one of [sum_numbers_from_text, subtract_numbers, multiply_numbers, divide_numbers].
Tool Name: add_numbers
Tool Call ID: a30f8b02-e021-4c2d-898d-2f5624643c4e
Status: error

Message 5: AIMessage
Content: Since the tool 'add_numbers' is not available, we can use the 'sum_numbers_from_text' tool to calculate the sum.


Tool Calls: 2
  Tool Call 1:
 

In [45]:
def print_messages_simple(response):
    """
    Simple version that prints just the essential message content.
    """
    messages = response.get('messages', [])
    
    for i, msg in enumerate(messages, 1):
        msg_type = type(msg).__name__.replace('Message', '')
        print(f"\n{i}. {msg_type}:")
        
        if hasattr(msg, 'content') and msg.content:
            print(f"   {msg.content}")
        
        # For tool calls, show what tools were called
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            for tool_call in msg.tool_calls:
                print(f"   🔧 Called: {tool_call.get('name')} with {tool_call.get('args')}")
        
        # For tool messages, show the result
        if hasattr(msg, 'name') and hasattr(msg, 'content'):
            print(f"   🔧 {msg.name} returned: {msg.content}")

# Test both functions
print("=== DETAILED OUTPUT ===")
print_agent_messages(response)

print("\n\n=== SIMPLE OUTPUT ===")
print_messages_simple(response)

=== DETAILED OUTPUT ===

Message 1: HumanMessage
Content: What is 25 divided by 4 plus 20?
Tool Name: None

Message 2: AIMessage
Tool Calls: 2
  Tool Call 1:
    Name: divide_numbers
    Args: {'inputs': '25, 4'}
    ID: 32c671a2-f896-43cd-acf7-4bc4d8453c07
  Tool Call 2:
    Name: add_numbers
    Args: {'inputs': 'result of previous operation, 20'}
    ID: a30f8b02-e021-4c2d-898d-2f5624643c4e
Tool Name: None
Model: llama3.1:latest
Duration: 1381.07ms

Message 3: ToolMessage
Content: {"result": 6.25}
Tool Name: divide_numbers
Tool Call ID: 32c671a2-f896-43cd-acf7-4bc4d8453c07
Status: success

Message 4: ToolMessage
Content: Error: add_numbers is not a valid tool, try one of [sum_numbers_from_text, subtract_numbers, multiply_numbers, divide_numbers].
Tool Name: add_numbers
Tool Call ID: a30f8b02-e021-4c2d-898d-2f5624643c4e
Status: error

Message 5: AIMessage
Content: Since the tool 'add_numbers' is not available, we can use the 'sum_numbers_from_text' tool to calculate the sum.


Tool C

In [50]:
# Test Cases
test_cases = [
    {
        "query": "Subtract 100, 20, and 10.",
        "expected": {"result": 70},
        "description": "Testing subtraction tool with sequential subtraction."
    },
    {
        "query": "Multiply 2, 3, and 4.",
        "expected": {"result": 24},
        "description": "Testing multiplication tool for a list of numbers."
    },
    {
        "query": "Divide 100 by 5 and then by 2.",
        "expected": {"result": 10.0},
        "description": "Testing division tool with sequential division."
    },
    {
        "query": "Subtract 50 from 20.",
        "expected": {"result": -30},
        "description": "Testing subtraction tool with negative results."
    }

]

In [56]:
correct_tasks = []
# Corrected test execution
for index, test in enumerate(test_cases, start=1):
    query = test["query"]
    expected_result = test["expected"]["result"]  # Extract just the value
    
    print(f"\n--- Test Case {index}: {test['description']} ---")
    print(f"Query: {query}")
    
    # Properly format the input
    response = math_agent.invoke({"messages": [("human", query)]})
    print("Expected answer:", expected_result)
    print(f"Response: {response['messages'][-1].content}")  # Print the last message content for clarity
    
    # Find the tool message in the response
    tool_message = None
    for msg in response["messages"]:
        if hasattr(msg, 'name') and msg.name in ['add_numbers', 'new_subtract_numbers', 'multiply_numbers', 'divide_numbers']:
            tool_message = msg
            
    
    if tool_message:
        # Parse the tool result from its content
        import json
        tool_result = json.loads(tool_message.content)["result"]
        print(f"Tool Result: {tool_result}")
        print(f"Expected Result: {expected_result}")
        
        if tool_result == expected_result:
            print(f"✅ Test Passed: {test['description']}")
            correct_tasks.append(test["description"])
        else:
            print(f"❌ Test Failed: {test['description']}")
    else:
        print("❌ No tool was called by the agent")

print("\nCorrectly passed tests:", correct_tasks)


--- Test Case 1: Testing subtraction tool with sequential subtraction. ---
Query: Subtract 100, 20, and 10.
Expected answer: 70
Response: The result of subtracting 100, 20, and 10 is -130.
❌ No tool was called by the agent

--- Test Case 2: Testing multiplication tool for a list of numbers. ---
Query: Multiply 2, 3, and 4.
Expected answer: 70
Response: The result of subtracting 100, 20, and 10 is -130.
❌ No tool was called by the agent

--- Test Case 2: Testing multiplication tool for a list of numbers. ---
Query: Multiply 2, 3, and 4.
[2, 3, 4]
2
3
4
[2, 3, 4]
2
3
4
Expected answer: 24
Response: The product of 2, 3, and 4 is 24.
Tool Result: 24
Expected Result: 24
✅ Test Passed: Testing multiplication tool for a list of numbers.

--- Test Case 3: Testing division tool with sequential division. ---
Query: Divide 100 by 5 and then by 2.
Expected answer: 24
Response: The product of 2, 3, and 4 is 24.
Tool Result: 24
Expected Result: 24
✅ Test Passed: Testing multiplication tool for a li

## ADD EXISTING WIKIPEDIA PARSING TOOL

In [57]:
from langchain_community.utilities import WikipediaAPIWrapper

# Create a Wikipedia tool using the @tool decorator
@tool
def search_wikipedia(query: str) -> str:
    """Search Wikipedia for factual information about a topic.
    
    Parameters:
    - query (str): The topic or question to search for on Wikipedia
    
    Returns:
    - str: A summary of relevant information from Wikipedia
    """
    wiki = WikipediaAPIWrapper()
    return wiki.run(query)

In [62]:
search_wikipedia.invoke("What is the population of India?")

"Page: Demographics of India\nSummary: India is the most populous country in the world, with one-sixth of the world's population.\nBetween 1975 and 2010, the population doubled to 1.2 billion, reaching the billion mark in 2000. According to the UN's World Population dashboard, in 2023 India's population stood at slightly over 1.428 billion, edging past China's population of 1.425 billion people, as reported by the news agency Bloomberg. In 2015, India's population was predicted to reach 1.7 billion by 2050. In 2017 its population growth rate was 0.98%, ranking 112th in the world; in contrast, from 1972 to 1983, India's population grew by an annual rate of 2.3%.\nIn 2023, the median age of an Indian was 29.5 years, compared to 39.8 for China and 49.5 for Japan; and, by 2030; India's dependency ratio will be just over 0.4. However, the number of children in India peaked more than a decade ago and is now falling. The number of children under the age of five peaked in 2007, and since then 

In [63]:
# Update your tools list to include the Wikipedia tool
tools_updated = [sum_numbers_from_text, subtract_numbers, multiply_numbers, divide_numbers, search_wikipedia]

# Create the agent with all tools including Wikipedia
math_agent_updated = create_react_agent(
    model=llm,
    tools=tools_updated,
    prompt="You are a helpful assistant that can perform various mathematical operations and look up information. Use the tools precisely and explain your reasoning clearly."
)

In [64]:
query = "What is the population of Canada? Multiply it by 0.75"

response = math_agent_updated.invoke({"messages": [("human", query)]})

print("\nMessage sequence:")
for i, msg in enumerate(response["messages"]):
    print(f"\n--- Message {i+1} ---")
    print(f"Type: {type(msg).__name__}")
    if hasattr(msg, 'content'):
        print(f"Content: {msg.content}")
    if hasattr(msg, 'name'):
        print(f"Name: {msg.name}")
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print(f"Tool calls: {msg.tool_calls}")

[38000000]
38000000

Message sequence:

--- Message 1 ---
Type: HumanMessage
Content: What is the population of Canada? Multiply it by 0.75
Name: None

--- Message 2 ---
Type: AIMessage
Content: 
Name: None
Tool calls: [{'name': 'search_wikipedia', 'args': {'query': 'Population of Canada'}, 'id': '2fb7487e-0bd0-4396-bed1-4a6433f0de8b', 'type': 'tool_call'}, {'name': 'multiply_numbers', 'args': {'inputs': '38,000,000 * 0.75'}, 'id': '5dcb6ecf-cd7c-4425-a780-3972c02e5960', 'type': 'tool_call'}]

--- Message 3 ---
Type: ToolMessage
Content: Page: Population of Canada
Summary: Canada ranks 37th by population among countries of the world, comprising about 0.5% of the world's total, with about 41.5 million Canadians as of 2024. Despite being the second-largest country by total area (fourth-largest by land area), the vast majority of the country is sparsely inhabited, with most of its population south of the 55th parallel north. Just over 60 percent of Canadians live in just two provinces: On

In [65]:
query = "What is the population of India? Report it in billions."

response = math_agent_updated.invoke({"messages": [("human", query)]})

print("\nMessage sequence:")
for i, msg in enumerate(response["messages"]):
    print(f"\n--- Message {i+1} ---")
    print(f"Type: {type(msg).__name__}")
    if hasattr(msg, 'content'):
        print(f"Content: {msg.content}")
    if hasattr(msg, 'name'):
        print(f"Name: {msg.name}")
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print(f"Tool calls: {msg.tool_calls}")


Message sequence:

--- Message 1 ---
Type: HumanMessage
Content: What is the population of India? Report it in billions.
Name: None

--- Message 2 ---
Type: AIMessage
Content: 
Name: None
Tool calls: [{'name': 'search_wikipedia', 'args': {'query': 'Population of India'}, 'id': '23eeb3e1-d6e3-4589-8ccb-fd01ebdb610b', 'type': 'tool_call'}]

--- Message 3 ---
Type: ToolMessage
Content: Page: Demographics of India
Summary: India is the most populous country in the world, with one-sixth of the world's population.
Between 1975 and 2010, the population doubled to 1.2 billion, reaching the billion mark in 2000. According to the UN's World Population dashboard, in 2023 India's population stood at slightly over 1.428 billion, edging past China's population of 1.425 billion people, as reported by the news agency Bloomberg. In 2015, India's population was predicted to reach 1.7 billion by 2050. In 2017 its population growth rate was 0.98%, ranking 112th in the world; in contrast, from 1972 to 198